In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras

/home/sfefilatyev/projects/tensorflow_exercises/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sfefilatyev/projects/tensorflow_exercises/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sfefilatyev/projects/tensorflow_exercises/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int1

In [2]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [3]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])
model.compile(loss="mean_squared_error", optimizer="sgd")
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3] # pretend these are new instances
y_pred = model.predict(X_new)
print(y_pred)

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 96us/sample - loss: 0.9051 - val_loss: 0.4902
Epoch 2/20
11610/11610 [==============================] - 0s 25us/sample - loss: 0.4894 - val_loss: 0.4394
Epoch 3/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.4515 - val_loss: 0.4236
Epoch 4/20
11610/11610 [==============================] - 0s 25us/sample - loss: 0.4413 - val_loss: 0.4153
Epoch 5/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.4624 - val_loss: 0.4243
Epoch 6/20
11610/11610 [==============================] - 0s 25us/sample - loss: 0.4344 - val_loss: 0.4137
Epoch 7/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.4241 - val_loss: 0.4086
Epoch 8/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.4178 - val_loss: 0.3981
Epoch 9/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.4131 - val_l

In [7]:
# Wide and deep neural net.
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model2 = keras.Model(inputs=[input_], outputs=[output])
model2.compile(loss="mean_squared_error", optimizer="sgd")
history = model2.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
mse_test = model2.evaluate(X_test, y_test)
X_new = X_test[:3] # pretend these are new instances
y_pred = model2.predict(X_new)
print(y_pred)

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 45us/sample - loss: 1.9083 - val_loss: 0.9354
Epoch 2/20
11610/11610 [==============================] - 0s 31us/sample - loss: nan - val_loss: nan
Epoch 3/20
11610/11610 [==============================] - 0s 28us/sample - loss: nan - val_loss: nan
Epoch 4/20
11610/11610 [==============================] - 0s 29us/sample - loss: nan - val_loss: nan
Epoch 5/20
11610/11610 [==============================] - 0s 28us/sample - loss: nan - val_loss: nan
Epoch 6/20
11610/11610 [==============================] - 0s 33us/sample - loss: nan - val_loss: nan
Epoch 7/20
11610/11610 [==============================] - 0s 34us/sample - loss: nan - val_loss: nan
Epoch 8/20
11610/11610 [==============================] - 0s 29us/sample - loss: nan - val_loss: nan
Epoch 9/20
11610/11610 [==============================] - 0s 27us/sample - loss: nan - val_loss: nan
Epoch 10/20
11610/11610 [===========